In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import os
from datetime import datetime
import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

In [3]:
#train_df = pd.read_csv('../../final/data/train_set_noisy_6_missing_3.csv')
train_df = pd.read_csv('../../final/data/train_set_basic.csv')
train_df = train_df.fillna(-1)
train_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,9.007012,-1.000000,-1.000000,3.519565,440.499323,103.442762,59.017997,2.612173,1,0.650757,114.794964,112.308159,25.612786,5.969710,116.026042,27.021037,13.397977,1
1,8.760976,-1.000000,0.491469,-1.000000,259.895852,103.885481,-1.000000,2.529991,0,0.728641,74.824352,-1.000000,-1.000000,-1.000000,-1.000000,26.282929,-1.000000,7
2,7.490324,70.812609,-1.000000,1.495604,482.109919,79.543391,-1.000000,2.824995,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,22.470972,-1.000000,4
3,8.113370,499.313483,3.507612,0.000000,195.351883,100.545858,203.895756,2.420797,1,0.860863,9.120947,41.131511,1.843890,18.845473,106.199806,24.340111,104.373581,2
4,13.935301,349.569415,5.190725,6.894195,489.595939,102.234294,150.085853,4.089225,1,0.216907,20.344863,92.547095,19.815123,29.543875,98.388710,41.805903,30.655044,0


In [4]:
test_df = pd.read_csv('../../final/data/test_set_constant.csv')
test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,7.116363,-1.000000,3.781573,2.738413,-1.000000,95.904198,68.457895,2.226085,0,1.892912,39.808550,110.329197,64.404350,21.654404,73.787009,21.349089,-1.000000,5
1,8.125320,92.230003,4.231419,1.188039,143.365567,104.057204,204.747831,2.342554,0,0.652614,13.478089,-1.000000,32.705481,-1.000000,43.520272,24.375961,142.815207,1
2,11.309450,38.324563,-1.000000,-1.000000,455.077909,76.402602,-1.000000,4.440732,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,33.928350,-1.000000,4
3,13.763858,253.513394,2.262606,0.551444,453.772884,82.781943,90.101466,4.987993,0,0.853521,104.005514,34.639227,0.963866,22.083012,88.891838,41.291574,19.856071,0
4,11.464002,-1.000000,-1.000000,-1.000000,320.964653,104.287127,-1.000000,3.297819,0,1.163516,121.616315,105.895897,-1.000000,9.337462,-1.000000,34.392007,-1.000000,7


In [5]:
len(train_df), len(test_df)

(56000, 14000)

In [6]:
action_list = list(constants.CLASS_DICT.keys()) + [col  for col in train_df.columns if col!='label']
action_list

['No anemia',
 'Vitamin B12/Folate deficiency anemia',
 'Unspecified anemia',
 'Anemia of chronic disease',
 'Iron deficiency anemia',
 'Hemolytic anemia',
 'Aplastic anemia',
 'Inconclusive diagnosis',
 'hemoglobin',
 'ferritin',
 'ret_count',
 'segmented_neutrophils',
 'tibc',
 'mcv',
 'serum_iron',
 'rbc',
 'gender',
 'creatinine',
 'cholestrol',
 'copper',
 'ethanol',
 'folate',
 'glucose',
 'hematocrit',
 'tsat']

In [7]:
train_df.isna().sum()

hemoglobin               0
ferritin                 0
ret_count                0
segmented_neutrophils    0
tibc                     0
mcv                      0
serum_iron               0
rbc                      0
gender                   0
creatinine               0
cholestrol               0
copper                   0
ethanol                  0
folate                   0
glucose                  0
hematocrit               0
tsat                     0
label                    0
dtype: int64

In [8]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]

X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]

X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

#### delete from here

#### end here

In [9]:
for steps in [int(6.5e6), int(7e6), int(7.5e6), int(8e6)]:
    dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, 
                                  f'../../final/models/new_experiments/logs/original_method/dqn_basic_{steps}')

using stable baselines 3
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.58     |
|    ep_rew_mean      | -0.94    |
|    exploration_rate | 0.553    |
|    success_rate     | 0.09     |
| time/               |          |
|    episodes         | 100000   |
|    fps              | 3529     |
|    time_elapsed     | 86       |
|    total_timesteps  | 306059   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 4.93     |
|    n_updates        | 64014    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.13     |
|    ep_rew_mean      | -1       |
|    exploration_rate | 0.05     |
|    success_rate     | 0.06     |
| time/               |          |
|    episodes         | 200000   |
|    fps              | 3019     |
|    t

using stable baselines 3
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.25     |
|    ep_rew_mean      | -0.76    |
|    exploration_rate | 0.588    |
|    success_rate     | 0.15     |
| time/               |          |
|    episodes         | 100000   |
|    fps              | 2784     |
|    time_elapsed     | 109      |
|    total_timesteps  | 303727   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 6.22     |
|    n_updates        | 63431    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.07     |
|    ep_rew_mean      | -0.96    |
|    exploration_rate | 0.0825   |
|    success_rate     | 0.07     |
| time/               |          |
|    episodes         | 200000   |
|    fps              | 2414     |
|    t

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.08     |
|    ep_rew_mean      | 0.48     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.79     |
| time/               |          |
|    episodes         | 1600000  |
|    fps              | 2054     |
|    time_elapsed     | 3215     |
|    total_timesteps  | 6606921  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.00567  |
|    n_updates        | 1639230  |
----------------------------------
using stable baselines 3
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.38     |
|    ep_rew_mean      | -0.82    |
|    exploration_rate | 0.617    |
|    success_rate     | 0.12     |
| time/               |          |
|    episodes         | 100000   |
|    fps              | 2977     |
|    t

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.07     |
|    ep_rew_mean      | 0.32     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.68     |
| time/               |          |
|    episodes         | 1500000  |
|    fps              | 2355     |
|    time_elapsed     | 2602     |
|    total_timesteps  | 6129606  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0667   |
|    n_updates        | 1519901  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.2      |
|    ep_rew_mean      | 0.24     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.69     |
| time/               |          |
|    episodes         | 1600000  |
|    fps              | 2337     |
|    time_elapsed     | 2804     |
|    total_timesteps  | 6555275  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.22     |
|    ep_rew_mean      | 0.46     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.78     |
| time/               |          |
|    episodes         | 1200000  |
|    fps              | 2492     |
|    time_elapsed     | 1955     |
|    total_timesteps  | 4874958  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.103    |
|    n_updates        | 1206239  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.39     |
|    ep_rew_mean      | 0.46     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.82     |
| time/               |          |
|    episodes         | 1300000  |
|    fps              | 738      |
|    time_elapsed     | 7192     |
|    total_timesteps  | 5308617  |
| train/              |          |
|    learning_rate  

In [ ]:
# training_env = utils.create_env(X_train, y_train)
dqn_model = utils.load_dqn3('../../models/many_features/0.1/with_correlated_fts/dqn3_by_type_noisy_4_missing_1_7500000', training_env)
test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
test_df.head()

In [ ]:
#utils.diagnose_sample(dqn_model, X_test, y_test, 1)

In [ ]:
# test_df[(test_df.y_pred==1) & (test_df.y_actual==1)]

In [ ]:
test_df.y_pred.value_counts()

In [ ]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

In [ ]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

In [ ]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

In [ ]:
test_df.y_pred.unique()

In [ ]:
# test_df.to_csv(f'../../test_dfs/many_features/0.1/test_df3_noisy_1_11000000.csv', index=False)
# success_df.to_csv(f'../../test_dfs/many_features/0.1/success_df3_noisy_1_11000000.csv', index=False)